DerivaML is a class library built on the Deriva Scientific Asset management system that is designed to help simplify a number of the basic operations associated with building and testing ML libraries based on common toolkits such as TensorFlow.  This notebook reviews the basic features of the DerivaML library.

In [1]:
from fontTools.subset import subset_features
%load_ext autoreload
%autoreload 2

In [2]:
import builtins
from deriva.core.utils.globus_auth_utils import GlobusNativeLogin
from deriva_ml.deriva_ml_base import ColumnDefinition, BuiltinTypes
from deriva_ml.schema_setup.test_catalog import create_test_catalog, DemoML
from deriva_ml.execution_configuration import ExecutionConfiguration, Workflow, Execution, WorkflowTerm, Term
import itertools
from pathlib import Path
import pandas as pd
import tempfile
import random

Set the details for the catalog we want and authenticate to the server if needed.

In [3]:
hostname = 'dev.eye-ai.org'
domain_schema = 'demo-schema'

gnl = GlobusNativeLogin(host=hostname)
if gnl.is_logged_in([hostname]):
    print("You are already logged in.")
else:
    gnl.login([hostname], no_local_server=True, no_browser=True, refresh_tokens=True, update_bdbag_keychain=True)
    print("Login Successful")


You are already logged in.


Create a test catalog and get an instance of the DerivaML class.

In [4]:
test_catalog = create_test_catalog(hostname, domain_schema)
ml_instance = DemoML(hostname, test_catalog.catalog_id)

2024-10-15 22:38:15,898 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2024-10-15 22:38:15,899 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>


A feature is a set of values that are attached to a table in the DerivaML catalog. Instances of features are distingushed from one another by the ID of the execution that produced the feature value. The execution could be the result of a program, or it could be a manual process by which a person defines a set of values

To create a new feature, we need to know the name of the feature, the table to which it is attached, and the set of values that make up the feature.  The values could be terms from a controlled vocabulary, a set of one or more file based assets, or other values, such as integers, or strings. However, use of strings outside of controlled vocabularies is discouraged.

In [5]:
# Prerequests for our feature, which will include a CV term and asset.

# Create a vocabulary and add a term to it to use in our features.
ml_instance.create_vocabulary("SubjectStatus", comment="A vocab for image quality")
ml_instance.add_term("SubjectStatus", "Sick", description="A sick subject")
ml_instance.add_term("SubjectStatus", "Well", description="A healthy subject")

box_asset = ml_instance.create_asset("BoundingBox", comment="A asset")

# Now lets create and upload a simple asset.
with tempfile.TemporaryDirectory() as tmpdirname:
    assetdir = Path(tmpdirname) / "BoundingBox"
    assetdir.mkdir(parents=True, exist_ok=True)
    for i in range(10):
        with builtins.open(assetdir / f"box{i}.txt", "w") as fp:
            fp.write(f"Hi there from box {i}")
    ml_instance.upload_assets(assetdir)

2024-10-15 22:38:22,750 - INFO - Initializing uploader: GenericUploader v1.7.4 [Python 3.12.3, macOS-15.0.1-x86_64-i386-64bit]
2024-10-15 22:38:22,751 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2024-10-15 22:38:22,751 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>
2024-10-15 22:38:22,903 - INFO - Checking for updated configuration...
2024-10-15 22:38:23,116 - INFO - Updated configuration found.
2024-10-15 22:38:23,119 - INFO - Scanning files in directory [/var/folders/0k/27qzm97x3t7g3j1m6ksf_9f40000gn/T/tmps2uksgx1/BoundingBox]...
2024-10-15 22:38:23,122 - INFO - Including file: [/var/folders/0k/27qzm97x3t7g3j1m6ksf_9f40000gn/T/tmps2uksgx1/BoundingBox/box7.txt].
2024-10-15 22:38:23,122 - INFO - Including file: [/var/folders/0k/27qzm97x3t7g3j1m6ksf_9f40000gn/T/tmps2uksgx1/BoundingBox/

In [6]:
# We are going to have three values in our feature, a controlled vocabulary term from the vocabulary FeatureValue, a file asset and 
# an integer value which we will call "TestCol"
ml_instance.create_feature("SubjectStatus", "Subject",
                                        terms=["SubjectStatus"],
                                        metadata=[ColumnDefinition(name='Scale', type=BuiltinTypes.int2)])

ml_instance.create_feature("BoundingBox", "Image",
                                        assets=[box_asset],
                                        metadata=[ColumnDefinition(name='Scale', type=BuiltinTypes.int2)])


print([f.feature_name for f in ml_instance.find_features("Subject")])
print([f.feature_name for f in ml_instance.find_features("Image")])


['SubjectStatus']
['BoundingBox']


Now we can add some features to our images.  To streamline the creation of new feature, we create a class that is specific to the arguments required to create it.

In [7]:
ImageFeatureClass = ml_instance.feature_record_class("Image", "BoundingBox")
SubjectFeatureClass = ml_instance.feature_record_class("Subject", "SubjectStatus")

print("Image Feature:")
display(ImageFeatureClass.model_fields)
print("Subject Feature:")
display(SubjectFeatureClass.model_fields)


Image Feature:


{'Execution': FieldInfo(annotation=str, required=True),
 'Image': FieldInfo(annotation=str, required=True),
 'Feature_Name': FieldInfo(annotation=str, required=False, default='BoundingBox'),
 'BoundingBox': FieldInfo(annotation=str, required=True),
 'Scale': FieldInfo(annotation=int, required=True)}

Subject Feature:


{'Execution': FieldInfo(annotation=str, required=True),
 'Subject': FieldInfo(annotation=str, required=True),
 'Feature_Name': FieldInfo(annotation=str, required=False, default='SubjectStatus'),
 'SubjectStatus': FieldInfo(annotation=str, required=True),
 'Scale': FieldInfo(annotation=int, required=True)}

Now using TestFeatureClass, we can create some instances of the feature and add it.  We must have a exeuction_rid in order to define the feature.

In [8]:
config = ExecutionConfiguration(
    execution=Execution(description="Sample Execution"), 
    workflow=Workflow(
        name="Sample Workflow", 
        url="https://github.com/informatics-isi-edu/deriva-ml/blob/main/pyproject.toml", 
        workflow_type="Sample Workflow"), 
    workflow_terms=[WorkflowTerm(term=Term.workflow, name="Sample Workflow", description="Sample Workflow Example")],
    description="Our Sample Workflow instance")
configuration_record = ml_instance.initialize_execution(config)
execution_rid = configuration_record.execution_rid

2024-10-15 22:38:42,105 - INFO - File [/var/folders/0k/27qzm97x3t7g3j1m6ksf_9f40000gn/T/tmppq7w0rpj/DemoML_working/exec_configiuje3r22.json] transfer successful. 0.45 KB transferred. Elapsed time: 0:00:00.000411.
2024-10-15 22:38:42,106 - INFO - Verifying MD5 checksum for downloaded file [/var/folders/0k/27qzm97x3t7g3j1m6ksf_9f40000gn/T/tmppq7w0rpj/DemoML_working/exec_configiuje3r22.json]
2024-10-15 22:38:42,232 - INFO - Configuration validation successful!


In [9]:
# Get some images to attach the feature value to.
image_rids = [i['RID'] for i in ml_instance.domain_path.tables['Image'].entities().fetch()]
asset_rids = [i['RID'] for i in ml_instance.domain_path.tables['BoundingBox'].entities().fetch()]
subject_rids = [i['RID'] for i in ml_instance.domain_path.tables['Subject'].entities().fetch()]

# Now create a list of features using the feature creation class returned by create_feature.
image_feature_list = [ImageFeatureClass(
    Image=image_rid,
    Execution=execution_rid,
    BoundingBox=asset_rid,
    Scale=random.randint(1, 10)) for image_rid, asset_rid in zip(image_rids, itertools.cycle(asset_rids))]

subject_feature_list = [SubjectFeatureClass(
    Subject=subject_rid,
    Execution=execution_rid,
    SubjectStatus=status,
    Scale=random.randint(1, 10)) for subject_rid, status in zip(subject_rids, itertools.cycle(["Sick", "Well"]))]

ml_instance.add_features(image_feature_list)
ml_instance.add_features(subject_feature_list)

20

In [10]:
system_columns = ['RCT', 'RMT', 'RCB', 'RMB']

display(pd.DataFrame(ml_instance.list_feature("Subject", "SubjectStatus")).drop(columns=system_columns))
display(pd.DataFrame(ml_instance.list_feature("Image", "BoundingBox")).drop(columns=system_columns))


,RID,Execution,Subject,Feature_Name,SubjectStatus,Scale
0,3EA,3CW,2ZG,SubjectStatus,Sick,4
1,3EC,3CW,2ZJ,SubjectStatus,Well,5
2,3EE,3CW,2ZM,SubjectStatus,Sick,7
3,3EG,3CW,2ZP,SubjectStatus,Well,7
4,3EJ,3CW,2ZR,SubjectStatus,Sick,4
5,3EM,3CW,2ZT,SubjectStatus,Well,7
6,3EP,3CW,2ZW,SubjectStatus,Sick,8
7,3ER,3CW,2ZY,SubjectStatus,Well,5
8,3ET,3CW,300,SubjectStatus,Sick,3
9,3EW,3CW,302,SubjectStatus,Well,8


,RID,Execution,Image,Feature_Name,BoundingBox,Scale
0,3D2,3CW,30R,BoundingBox,36A,7
1,3D4,3CW,30T,BoundingBox,36C,7
2,3D6,3CW,30W,BoundingBox,36E,2
3,3D8,3CW,30Y,BoundingBox,36G,3
4,3DA,3CW,310,BoundingBox,36J,3
5,3DC,3CW,312,BoundingBox,36M,7
6,3DE,3CW,314,BoundingBox,36P,10
7,3DG,3CW,316,BoundingBox,36R,5
8,3DJ,3CW,318,BoundingBox,36T,5
9,3DM,3CW,31A,BoundingBox,36W,4


## Computing Features

In the previous example, we used the API to create new features. However, creating new feature values is also integrated into the process fo managing an execution in deriva-ml.

First we will create a new execution for our computation.

In [11]:
config = ExecutionConfiguration(
    execution=Execution(description="Our second execution"),
    workflow=Workflow(
        name="Sample Workflow",
        url="https://github.com/informatics-isi-edu/deriva-ml/blob/main/pyproject.toml",
        workflow_type="Sample Workflow"),
    workflow_terms=[WorkflowTerm(term=Term.workflow, name="Sample Workflow", description="Sample Workflow Example")],
    description="Our Sample Workflow instance")
configuration_record = ml_instance.initialize_execution(config)
execution_rid = configuration_record.execution_rid

2024-10-15 22:39:05,788 - INFO - File [/var/folders/0k/27qzm97x3t7g3j1m6ksf_9f40000gn/T/tmppq7w0rpj/DemoML_working/exec_configme_jwxjx.json] transfer successful. 0.46 KB transferred. Elapsed time: 0:00:00.000485.
2024-10-15 22:39:05,789 - INFO - Verifying MD5 checksum for downloaded file [/var/folders/0k/27qzm97x3t7g3j1m6ksf_9f40000gn/T/tmppq7w0rpj/DemoML_working/exec_configme_jwxjx.json]
2024-10-15 22:39:05,914 - INFO - Configuration validation successful!


In [12]:
ml_instance.add_dataset_element_type("Subject")
ml_instance.add_term("Dataset_Type", "Subject Dataset", description="Subject Dataset")
subject_dataset = ml_instance.create_dataset("Subject Dataset", description="Subject Dataset")
ml_instance.add_dataset_members(subject_dataset, subject_rids)

'3J2'

In [13]:
bag_path = ml_instance.materialize_dataset_bag(subject_dataset)[0]

2024-10-15 22:39:19,216 - INFO - Initializing downloader: GenericDownloader v1.7.4 [Python 3.12.3, macOS-15.0.1-x86_64-i386-64bit]
2024-10-15 22:39:19,218 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2024-10-15 22:39:19,218 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>
2024-10-15 22:39:19,221 - INFO - Validating credentials for host: dev.eye-ai.org
2024-10-15 22:39:19,367 - INFO - Creating bag directory: /var/folders/0k/27qzm97x3t7g3j1m6ksf_9f40000gn/T/tmphwqkl14u/Dataset_3J2
2024-10-15 22:39:19,370 - INFO - Creating bag for directory /var/folders/0k/27qzm97x3t7g3j1m6ksf_9f40000gn/T/tmphwqkl14u/Dataset_3J2
2024-10-15 22:39:19,371 - INFO - Creating data directory
2024-10-15 22:39:19,372 - INFO - Moving /private/var/folders/0k/27qzm97x3t7g3j1m6ksf_9f40000gn/T/tmphwqkl14u/Dataset_3J2/tmp

In [14]:
bag_path

PosixPath('/private/var/folders/0k/27qzm97x3t7g3j1m6ksf_9f40000gn/T/tmppq7w0rpj/3J2_913efdd6752999ef81e9955d65c236072e3ea949ae4bc58598c1472ba59c1325/Dataset_3J2')

In [ ]:
test_catalog.delete_ermrest_catalog(really=True)